## Heilbrigðislíkan
við erum

In [2]:
import simpy as sp
import streamlit as st
import numpy as np
import math
import random

In [11]:
class Manneskja:
    def __init__(self,aldur):
        self.aldur = aldur
        if self.aldur < 65:
            self.prob = 0.02
        else:
            self.prob = 0.1
    def MtoP(self):
        p = Patient(self.aldur,"móttaka")
        return p

class Patient:
    def __init__(self,aldur,place):
        self.aldur = aldur
        self.place = place
        if place == "skurðaaðgerð":
            self.p_death = 0.1
            self.p_surgery = 0.0
            self.p_mottaka = 0.9
        if place == "móttaka":
            self.p_death = 0.0
            self.p_home = 0.7
            self.p_surgery = 0.3
    def PtoM(self):
        m = Manneskja(self.aldur)
        return m

In [15]:
class Spitali:
    def __init__(self,cap):
        self.cap = cap
        self.amount = 0
    def addP(self):
        if self.amount == self.cap:
            raise Exception("Nú er spítalinn fullur það tók")
        self.amount += 1
    def removeP(self):
        self.amount -= 1

In [17]:
N = 1000
t = 1
timi = 0
P = []
M = []
S = Spitali(500)
fj_surg = 0
fj_mot = 0
for p in range(0,299):
    age = random.randint(65,99)
    pi = Manneskja(age)
    M.append(pi)
for p in range(300,N):
    age = random.randint(1,64)
    pi = Manneskja(age)
    M.append(pi)
while(True):
    timi += t
    for m in M:
        r = random.random()
        if r < m.prob:
            S.addP()
            pi = m.MtoP()
            M.remove(m)
            P.append(pi)
    for p in P:
        r = random.random()
        if p.place == "móttaka":
            if r < p.p_death:
                P.remove(p)
                S.removeP()
            if r >= p.p_death and r <= p.p_surgery:
                P.remove(p)
                S.removeP()
                m = p.PtoM()
                M.append(m)
            else:
                p.place = "skurðaaðgerð"
        if p.place == "skurðaaðgerð":
            if r < p.p_surgery:
                p.place = "skurðaaðgerð"
            if r >= p.p_surgery and r <= p.p_death:
                P.remove(p)
                S.removeP()
            else:
                p.place = "móttaka"
    print("fjöldi fólks á spítala á degi",timi,"er",S.amount)

fjöldi fólks á spítala á degi 1 er 28
fjöldi fólks á spítala á degi 2 er 53
fjöldi fólks á spítala á degi 3 er 74
fjöldi fólks á spítala á degi 4 er 84
fjöldi fólks á spítala á degi 5 er 89
fjöldi fólks á spítala á degi 6 er 85
fjöldi fólks á spítala á degi 7 er 95
fjöldi fólks á spítala á degi 8 er 106
fjöldi fólks á spítala á degi 9 er 98
fjöldi fólks á spítala á degi 10 er 115
fjöldi fólks á spítala á degi 11 er 107
fjöldi fólks á spítala á degi 12 er 107
fjöldi fólks á spítala á degi 13 er 104
fjöldi fólks á spítala á degi 14 er 96
fjöldi fólks á spítala á degi 15 er 96
fjöldi fólks á spítala á degi 16 er 98
fjöldi fólks á spítala á degi 17 er 112
fjöldi fólks á spítala á degi 18 er 110
fjöldi fólks á spítala á degi 19 er 120
fjöldi fólks á spítala á degi 20 er 121
fjöldi fólks á spítala á degi 21 er 119
fjöldi fólks á spítala á degi 22 er 111
fjöldi fólks á spítala á degi 23 er 110
fjöldi fólks á spítala á degi 24 er 106
fjöldi fólks á spítala á degi 25 er 97
fjöldi fólks á spítal

KeyboardInterrupt: 